In [1]:
!pip install fastparquet

Sem Pyspark:


In [2]:
import os
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Função para carregar e pré-processar os dados
def load_and_preprocess_data(file_path):
    print("Iniciando o carregamento e pré-processamento dos dados...")
    start_time = time.time()

    # Carregar o arquivo Parquet consolidado
    data = pd.read_parquet(file_path, engine="fastparquet")

    # Filtrar classes relevantes (0 a 9)
    valid_classes = list(range(10))
    if 'class' in data.columns:
        data = data[data['class'].isin(valid_classes)]

    # Separar features e rótulos
    X = data.drop(columns=['class', 'state']).dropna()
    y = data.loc[X.index, 'class']

    # Normalizar as features
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)

    elapsed_time = time.time() - start_time
    print(f"Tempo de carregamento e pré-processamento: {elapsed_time:.2f} segundos")
    return X_normalized, y

# Função para treinar o modelo Random Forest
def train_random_forest(X, y):
    print("Iniciando o treinamento do modelo Random Forest...")
    start_time = time.time()

    # Dividir os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Definir o modelo Random Forest com hiperparâmetros ajustados
    model = RandomForestClassifier(
        n_estimators=100,  # Número de árvores na floresta
        max_depth=10,      # Profundidade máxima das árvores
        random_state=42,   # Reprodutibilidade
        n_jobs=-1          # Usar todos os núcleos disponíveis
    )

    # Treinar o modelo
    model.fit(X_train, y_train)

    # Fazer previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Avaliar o modelo
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    macro_f1_score = f1_score(y_test, y_pred, average='macro')

    elapsed_time = time.time() - start_time
    print(f"Tempo de treinamento e avaliação: {elapsed_time:.2f} segundos")
    return model, balanced_accuracy, macro_f1_score

# Medir tempo total de execução do script
overall_start_time = time.time()

# Caminho do arquivo consolidado
input_file = "/content/dados_limpos1.parquet"

# Carregar e pré-processar os dados
X, y = load_and_preprocess_data(input_file)

# Treinar e avaliar o modelo
model, balanced_accuracy, macro_f1_score = train_random_forest(X, y)

# Tempo total do script
overall_elapsed_time = time.time() - overall_start_time

# Exibir métricas finais
print(f"Tempo total de execução: {overall_elapsed_time:.2f} segundos")
print(f"Acurácia Balanceada: {balanced_accuracy:.4f}")
print(f"Macro F1-Score: {macro_f1_score:.4f}")




Iniciando o carregamento e pré-processamento dos dados...
Tempo de carregamento e pré-processamento: 41.04 segundos
Iniciando o treinamento do modelo Random Forest...
Tempo de treinamento e avaliação: 7456.80 segundos
Tempo total de execução: 7497.99 segundos
Acurácia Balanceada: 0.8709
Macro F1-Score: 0.8716


Com Pyspark:

In [3]:
!pip install pyspark


In [4]:

import os
import time
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Configurar a sessão do Spark para o Colab
spark = (
    SparkSession.builder
    .appName("PySparkRandomForestColab")
    .config("spark.executor.memory", "11g")  # Ajustar memória do executor
    .config("spark.driver.memory", "8g")  # Ajustar memória do driver
    .config("spark.sql.shuffle.partitions", "12")  # Ajustar partições
    .getOrCreate()
)

# Função para carregar e pré-processar os dados
def load_and_preprocess_data(file_path):
    start_time = time.time()
    print("Iniciando o carregamento e pré-processamento dos dados...")

    # Carregar os dados usando PySpark
    data = spark.read.parquet(file_path)

    # Filtrar classes relevantes
    data = data.filter(data["class"].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

    # Montar o vetor de features
    feature_columns = [col for col in data.columns if col not in ["class", "state"]]
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    data = assembler.transform(data)

    # Normalizar as features
    scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
    scaler_model = scaler.fit(data)
    data = scaler_model.transform(data)

    elapsed_time = time.time() - start_time
    print(f"Carregamento e pré-processamento concluídos. Tempo: {elapsed_time:.2f} segundos")
    return data, elapsed_time

# Função para treinar o modelo Random Forest
def train_random_forest(data):
    start_time = time.time()
    print("Iniciando o treinamento do modelo Random Forest...")

    # Dividir os dados em treino e teste
    train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

    # Treinar o modelo Random Forest
    rf = RandomForestClassifier(labelCol="class", featuresCol="scaledFeatures", numTrees=100, maxDepth=10)
    model = rf.fit(train_data)

    # Fazer previsões no conjunto de teste
    predictions = model.transform(test_data)

    # Avaliar o modelo
    evaluator = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction", metricName="weightedPrecision")
    balanced_accuracy = evaluator.evaluate(predictions)

    evaluator_macro_f1 = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction", metricName="f1")
    macro_f1_score = evaluator_macro_f1.evaluate(predictions)

    elapsed_time = time.time() - start_time
    print(f"Treinamento e avaliação concluídos. Tempo: {elapsed_time:.2f} segundos")
    print(f"Acurácia Balanceada: {balanced_accuracy:.4f}")
    print(f"Macro F1-Score: {macro_f1_score:.4f}")
    return model, elapsed_time, balanced_accuracy, macro_f1_score

# Medir tempo total
overall_start_time = time.time()

# Caminho do arquivo de entrada
input_file = "/content/dados_limpos1.parquet"

# Etapa 1: Carregar e pré-processar os dados
data, preprocessing_time = load_and_preprocess_data(input_file)

# Etapa 2: Treinar e avaliar o modelo
model, training_time, balanced_accuracy, macro_f1_score = train_random_forest(data)

# Exibir tempo total de execução
overall_elapsed_time = time.time() - overall_start_time
print("\nResumo de execução:")
print(f"Tempo de carregamento e pré-processamento: {preprocessing_time:.2f} segundos")
print(f"Tempo de treinamento e avaliação: {training_time:.2f} segundos")
print(f"Tempo total de execução: {overall_elapsed_time:.2f} segundos")
print(f"Acurácia Balanceada: {balanced_accuracy:.4f}")
print(f"Macro F1-Score: {macro_f1_score:.4f}")

# Finalizar o Spark
spark.stop()
print("Execução concluída.")



Iniciando o carregamento e pré-processamento dos dados...
Carregamento e pré-processamento concluídos. Tempo: 369.47 segundos
Iniciando o treinamento do modelo Random Forest...
Treinamento e avaliação concluídos. Tempo: 10455.26 segundos
Acurácia Balanceada: 0.9933
Macro F1-Score: 0.9928

Resumo de execução:
Tempo de carregamento e pré-processamento: 369.47 segundos
Tempo de treinamento e avaliação: 10455.26 segundos
Tempo total de execução: 10824.74 segundos
Acurácia Balanceada: 0.9933
Macro F1-Score: 0.9928
Execução concluída.


In [ ]:
Conclusões finais:

O uso do PySpark resultou em um modelo significativamente mais preciso, com ganhos de 14,05% na acurácia balanceada e 13,90% no Macro F1-Score. Isso provavelmente se deve ao paralelismo interno do RandomForestClassifier no Spark, que ajuda a capturar padrões mais robustos nos dados.


Também podemos notar aumentou o tempo de execução em aproximadamente 44% com o uso do PySpark, provavelmente devido ao overhead de configuração e execução distribuída no Spark. Para datasets pequenos ou médios, o PySpark pode ser menos eficiente em relação à execução direta.


Para o caso estudado aqui podemos notar um claro trade-off: maior desempenho em clasificação ao custo de um tempo de execução maior.
